# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df = pd.read_csv('../wheather_dataframe.csv')
weather_df.head()

,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2122090.0,Pevek,RU,69.70,170.31,07/20/20,46.54,59.0,8.0,2.64
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,244878.0,Biltine,TD,14.53,20.92,07/20/20,75.38,70.0,99.0,5.82
4,1015776.0,Bredasdorp,ZA,-34.53,20.04,07/20/20,60.80,93.0,25.0,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:

# Store latitude and longitude in locations
lat_lon = weather_df[['latitude', 'longitude']]

# Fill NaN values and convert to float
humid = weather_df["humidity"]
lat_lon['latitude'].value_counts()
nanindex = lat_lon.loc[lat_lon['latitude'].isnull()].index
#lat_lon['latitude'].isnull().values.sum()
weather_df.drop(nanindex,inplace=True)
weather_df

,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed
1,2122090.0,Pevek,RU,69.70,170.31,07/20/20,46.54,59.0,8.0,2.64
3,244878.0,Biltine,TD,14.53,20.92,07/20/20,75.38,70.0,99.0,5.82
4,1015776.0,Bredasdorp,ZA,-34.53,20.04,07/20/20,60.80,93.0,25.0,10.29
5,5380437.0,Pacific Grove,US,36.62,-121.92,07/20/20,63.61,68.0,1.0,12.75
6,5848280.0,Kapaa,US,22.08,-159.32,07/20/20,83.16,62.0,1.0,14.99
...,...,...,...,...,...,...,...,...,...,...
621,2013279.0,Vostok,RU,46.49,135.88,07/20/20,60.26,100.0,100.0,1.16
624,3137469.0,Sørland,NO,67.67,12.69,07/20/20,53.58,77.0,82.0,6.69
625,3454818.0,Pato Branco,BR,-26.23,-52.67,07/20/20,57.38,89.0,10.0,3.65
627,2636790.0,Stornoway,GB,58.21,-6.39,07/20/20,46.00,100.0,20.0,4.70


In [7]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at
# a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
# Create heat layer
heat_layer = gmaps.heatmap_layer(lat_lon, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# A max temperature lower than 80 degrees but higher than 70.
crit_temperature = (weather_df.temp < 80) & (weather_df.temp > 70)
crit_windSpeed = weather_df.wind_speed < 15
crit_cloudiness = weather_df.cloudiness == 0
final_criteria = crit_temperature & crit_windSpeed & crit_cloudiness 

# Use boolean indexing to filter the weather_df dataframe
ideal_weather_df = weather_df[final_criteria]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df.head(10)

,index,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed
0,46,4032243.0,Vaini,TO,-21.20,-175.20,07/20/20,75.20,64.0,0.0,8.05
1,129,1630639.0,Pringsewu,ID,-5.36,104.97,07/20/20,74.39,91.0,0.0,0.60
2,130,135423.0,Shahrud,IR,36.42,54.98,07/20/20,72.27,27.0,0.0,5.08
3,156,140463.0,Birjand,IR,32.87,59.22,07/20/20,75.20,23.0,0.0,9.17
4,157,2214846.0,Misratah,LY,32.38,15.09,07/20/20,76.89,78.0,0.0,9.64
5,178,86049.0,Jalu,LY,29.03,21.55,07/20/20,77.14,50.0,0.0,9.42
6,184,124665.0,Mashhad,IR,36.30,59.61,07/20/20,77.00,27.0,0.0,6.93
7,348,1646170.0,Cirebon,ID,-6.71,108.56,07/20/20,79.12,77.0,0.0,6.71
8,379,2215163.0,Masallātah,LY,32.62,14.00,07/20/20,70.95,84.0,0.0,6.20
9,410,362973.0,Abnūb,EG,27.27,31.15,07/20/20,78.80,34.0,0.0,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = ideal_weather_df
# Add column for Hotel Name - Note that we used "" to specify initial entry.
hotel_df['Hotel Name'] = ""
hotel_df.head()

,index,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed,Hotel Name
0,46,4032243.0,Vaini,TO,-21.20,-175.20,07/20/20,75.20,64.0,0.0,8.05,
1,129,1630639.0,Pringsewu,ID,-5.36,104.97,07/20/20,74.39,91.0,0.0,0.60,
2,130,135423.0,Shahrud,IR,36.42,54.98,07/20/20,72.27,27.0,0.0,5.08,
3,156,140463.0,Birjand,IR,32.87,59.22,07/20/20,75.20,23.0,0.0,9.17,
4,157,2214846.0,Misratah,LY,32.38,15.09,07/20/20,76.89,78.0,0.0,9.64,


In [15]:
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        #hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #hotel_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df.info()

Missing field/result... skipping.
Missing field/result... skipping.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         17 non-null     int64  
 1   city_id       17 non-null     float64
 2   city_name     17 non-null     object 
 3   country       17 non-null     object 
 4   latitude      17 non-null     float64
 5   longitude     17 non-null     float64
 6   weather_date  17 non-null     object 
 7   temp          17 non-null     float64
 8   humidity      17 non-null     float64
 9   cloudiness    17 non-null     float64
 10  wind_speed    17 non-null     float64
 11  Hotel Name    17 non-null     object 
dtypes: float64(7), int64(1), object(4)
memory usage: 1.7+ KB


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>city_name</dt><dd>{city_name}</dd>
<dt>country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [13]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…